In [33]:
import sys, os; sys.path.append('..')
import pyzx as zx
import random
import math
from fractions import Fraction
%config InlineBackend.figure_format = 'svg'

In [35]:
c = zx.qasm("""
qreg q[3];
cx q[0], q[1];
""")
zx.d3.draw(c)

In [36]:
c = zx.qasm("""
qreg q[2];
rx(0.5*pi) q[1];
t q[0];
cx q[0], q[1];
cx q[1], q[0];
cx q[0], q[1];
tdg q[1];
rx(-0.5*pi) q[0];
""")
zx.d3.draw(c)

In [37]:
c.gates

[XPhase(1,phase=1/2),
 T(0),
 CNOT(0,1),
 CNOT(1,0),
 CNOT(0,1),
 T*(1),
 XPhase(0,phase=-1/2)]

In [38]:
g = c.to_graph()
g

Graph(14 vertices, 15 edges)

In [39]:
zx.simplify.spider_simp(g)
zx.d3.draw(g)

spider_simp: 1. 1.  2 iterations


In [40]:
zx.full_reduce(g)
zx.d3.draw(g)

In [43]:
g = zx.sqasm("""
qreg S[1];
qreg q[3];
t S[0];
cx q[0], S[0];
cx q[1], S[0];
cx q[2], S[0];
""")
zx.d3.draw(g)

In [44]:
g = zx.sqasm("""
qreg S[2];
qreg q[3];
t S[0];
cx q[0], S[0];
cx q[1], S[0];
cx q[2], S[0];
tdg S[1];
cx q[0], S[1];
cx q[1], S[1];
cx q[2], S[1];
""")
zx.d3.draw(g)

In [ ]:
zx.clifford_simp(g)
zx.d3.draw(g)

In [45]:
zx.full_reduce(g)
zx.d3.draw(g)

In [46]:
g = zx.Circuit.load("test.qsim").to_graph()
zx.d3.draw(g)

In [47]:
g1 = g.copy()
g1.map_qubits([
    (0,0), (1, 0), (2, 0), (3, 0),
    (0,1), (1, 1), (2, 1), (3, 1),
    (0,2), (1, 2), (2, 2), (3, 2),
    (0,3), (1, 3), (2, 3), (3, 3)
])
zx.d3.draw(g1)

In [48]:
zx.full_reduce(g1)
zx.d3.draw(g1)

In [ ]:
f = open('demo.graphml', 'w')
f.write(zx.to_graphml(g1))
f.close()

In [ ]:
def t_optimiser(c):
    g = c.to_graph()
    g = zx.simplify.teleport_reduce(g)
    c_opt = zx.Circuit.from_graph(g).split_phase_gates().to_basic_gates()
    return zx.optimize.basic_optimization(c_opt).to_basic_gates()

In [49]:
c = zx.Circuit.load('../circuits/Fast/grover_5.qc')
zx.d3.draw(c.to_graph())
print(zx.tcount(c))

336


In [50]:
c1 = t_optimiser(c)
zx.d3.draw(c1.to_graph())
print(zx.tcount(c1))

166


In [51]:
c.verify_equality(c1)

True

In [52]:
c2 = c1.copy()
c2.add_circuit(c.adjoint())
g = c2.to_graph()
zx.simplify.full_reduce(g)
zx.d3.draw(g)

In [53]:
c1.gates[10] = zx.gates.T(6, adjoint=True)

In [54]:
c.verify_equality(c1)

False

In [ ]:
c2 = c1.copy()
c2.add_circuit(c.adjoint())
g = c2.to_graph()
zx.simplify.full_reduce(g)
zx.d3.draw(g)

In [55]:
g = zx.Circuit.load('../circuits/Fast/hwb6.qc').to_graph()
zx.d3.draw(g)
print(zx.tcount(g))

105


In [ ]:
zx.simplify.full_reduce(g)
zx.d3.draw(g)
print(zx.tcount(g))

In [56]:
g.apply_state("++---+-")
g.apply_effect("+011-1-")
zx.simplify.full_reduce(g)
print(zx.tcount(g))
zx.drawing.arrange_scalar_diagram(g)
zx.d3.draw(g)

33


In [57]:
def compute_decomp(g):
    if zx.tcount(g) >= 6:
        gsum = zx.simulate.replace_magic_states(g)
        gsum.reduce_scalar()
        terms = 0
        vals = 0
        for g1 in gsum.graphs:
            t,v = compute_decomp(g1)
            terms += t
            vals += v
        return (terms, vals)
    else:
        return (2 ** math.ceil(zx.tcount(g)/2), g.to_matrix())

In [58]:
math.ceil(2**(0.468 * zx.tcount(g)))

44577

In [59]:
compute_decomp(g)

(10688, array([[-0.0625-0.j]]))

In [ ]:
(2**12, zx.simulate.calculate_path_sum(g))